***Connect to google drive colab***

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


***Installation of dependencies***

In [ ]:
!pip install assemblyai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.6/72.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00


In [ ]:
!pip install deep_translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.1 MB/s eta 0:00:00


***Display audio***

In [ ]:
from IPython.display import Audio

# Function to handle audio playback
def play_audio(file_path, rate=None):
    """Plays an audio file.

    Args:
        file_path (str): The path to the audio file.
        rate (int, optional): The sample rate of the audio.
                               If None, attempts to autodetect.
                               Defaults to None.
    """
    # If rate is not provided, assume it's an audio file and let Audio autodetect
    # If rate is provided, assume data is numpy array and pass rate
    audio = Audio(filename=file_path, rate=44100)
    display(audio)

audio_file = "/content/drive/MyDrive/AI_ML_INTERN /Task 3 Practice /vitals1.mp3"
play_audio(audio_file)

Output hidden; open in https://colab.research.google.com to view.

***Speech recognation***

In [ ]:
import requests
import json
import time

def read_file(filename, chunk_size=5242880):
    with open(filename, 'rb') as _file:
        while True:
            data = _file.read(chunk_size)
            if not data:
                break
            yield data

def upload_file(api_token, path):
    """
    Upload a file to the AssemblyAI API.

    Args:
        api_token (str): Your API token for AssemblyAI.
        path (str): Path to the local file.

    Returns:
        str: The upload URL.
    """
    headers = {'authorization': api_token}
    response = requests.post('https://api.assemblyai.com/v2/upload',
                             headers=headers,
                             data=read_file(path))

    if response.status_code == 200:
        return response.json()["upload_url"]
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

def create_transcript(api_token, audio_url):
    """
    Create a transcript using AssemblyAI API.

    Args:
        api_token (str): Your API token for AssemblyAI.
        audio_url (str): URL of the audio file to be transcribed.

    Returns:
        dict: Completed transcript object.
    """
    # Set the API endpoint for creating a new transcript
    url = "https://api.assemblyai.com/v2/transcript"

    # Set the headers for the request, including the API token and content type
    headers = {
        "authorization": api_token,
        "content-type": "application/json"
    }

    # Set the data for the request, including the URL of the audio file to be transcribed
    data = {
        "audio_url": audio_url
    }

    # Send a POST request to the API to create a new transcript, passing in the headers and data
    response = requests.post(url, json=data, headers=headers)

    # Get the transcript ID from the response JSON data
    transcript_id = response.json()['id']

    # Set the polling endpoint URL by appending the transcript ID to the API endpoint
    polling_endpoint = f"https://api.assemblyai.com/v2/transcript/{transcript_id}"

    # Keep polling the API until the transcription is complete
    while True:
        # Send a GET request to the polling endpoint, passing in the headers
        transcription_result = requests.get(polling_endpoint, headers=headers).json()

        # If the status of the transcription is 'completed', exit the loop
        if transcription_result['status'] == 'completed':
            break

        # If the status of the transcription is 'error', raise a runtime error with the error message
        elif transcription_result['status'] == 'error':
            raise RuntimeError(f"Transcription failed: {transcription_result['error']}")

        # If the status of the transcription is not 'completed' or 'error', wait for 3 seconds and poll again
        else:
            time.sleep(3)

    return transcription_result

your_api_token = "bc7e213c3d14435d9f1e198ce37badb2"

# Upload a local file
filename = "/content/drive/MyDrive/AI_ML_INTERN /Task 3 Practice /vitals1.mp3"
upload_url = upload_file(your_api_token, filename)

# Transcribe it
transcript = create_transcript(your_api_token, upload_url)

# Print the completed transcript object
result=transcript['text']
print(result)

Good morning, Mr. Anderson. How are you feeling today Morning? I'm feeling a bit tired, but overall not too bad. Alright, I'm just going to take a few of your vital signs. I'll start with your blood pressure. Could you roll up your sleeves please? Sure thing. Thank you. Okay, just stay relaxed. Here we go. Your blood pressure is 120 over 80, which is in a good range. Now let me check your pulse. I'll place my finger here on your wrist. No problem. That's a pulse of 72 beats per minute, which is within the normal range. Now I'll check your temperature. I'll place this thermometer under your tongue so that please keep your mouth closed around it. Great. Your temperature is 98.6 degrees Fahrenheit. Right on the target. Finally, I'll check your oxygen levels with this little device. I'll just clip it on your finger. Okay? Perfect. Your oxygen level is at 98%. Everything looks good. Med standards in. Let me know if you start feeling any discomfort or have any concerns. Thank you. I apprecia

In [ ]:
print(transcript)

{'id': '4f22b272-cd88-4747-a7f6-6ed5cadab6e9', 'language_model': 'assemblyai_default', 'acoustic_model': 'assemblyai_default', 'language_code': 'en_us', 'status': 'completed', 'audio_url': 'https://cdn.assemblyai.com/upload/35c6cb08-b9bd-481e-9682-3cbbb0f67397', 'text': "Good morning, Mr. Anderson. How are you feeling today Morning? I'm feeling a bit tired, but overall not too bad. Alright, I'm just going to take a few of your vital signs. I'll start with your blood pressure. Could you roll up your sleeves please? Sure thing. Thank you. Okay, just stay relaxed. Here we go. Your blood pressure is 120 over 80, which is in a good range. Now let me check your pulse. I'll place my finger here on your wrist. No problem. That's a pulse of 72 beats per minute, which is within the normal range. Now I'll check your temperature. I'll place this thermometer under your tongue so that please keep your mouth closed around it. Great. Your temperature is 98.6 degrees Fahrenheit. Right on the target. Fi

'audio_url': 'https://cdn.assemblyai.com/upload/35c6cb08-b9bd-481e-9682-3cbbb0f67397'

In [ ]:
# Split the transcript into chunks of maximum length 80 characters
chunks = [transcript['text'][i:i+110] for i in range(0, len(transcript['text']), 110)]

# Join the chunks with line breaks
formatted_text = "\n".join(chunks)

print(formatted_text)

Good morning, Mr. Anderson. How are you feeling today Morning? I'm feeling a bit tired, but overall not too ba
d. Alright, I'm just going to take a few of your vital signs. I'll start with your blood pressure. Could you r
oll up your sleeves please? Sure thing. Thank you. Okay, just stay relaxed. Here we go. Your blood pressure is
 120 over 80, which is in a good range. Now let me check your pulse. I'll place my finger here on your wrist. 
No problem. That's a pulse of 72 beats per minute, which is within the normal range. Now I'll check your tempe
rature. I'll place this thermometer under your tongue so that please keep your mouth closed around it. Great. 
Your temperature is 98.6 degrees Fahrenheit. Right on the target. Finally, I'll check your oxygen levels with 
this little device. I'll just clip it on your finger. Okay? Perfect. Your oxygen level is at 98%. Everything l
ooks good. Med standards in. Let me know if you start feeling any discomfort or have any concerns. Thank you. 
I

***Speaker Diarization***

In [ ]:
import requests
import json
import time

def read_file(filename, chunk_size=5242880):
    with open(filename, 'rb') as _file:
        while True:
            data = _file.read(chunk_size)
            if not data:
                break
            yield data

def upload_file(api_token, path):
    """
    Upload a file to the AssemblyAI API.

    Args:
        api_token (str): Your API token for AssemblyAI.
        path (str): Path to the local file.

    Returns:
        str: The upload URL.
    """
    headers = {'authorization': api_token}
    response = requests.post('https://api.assemblyai.com/v2/upload',
                             headers=headers,
                             data=read_file(path))

    if response.status_code == 200:
        return response.json()["upload_url"]
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

def create_transcript(api_token, audio_url):
    """
    Create a transcript using AssemblyAI API.

    Args:
        api_token (str): Your API token for AssemblyAI.
        audio_url (str): URL of the audio file to be transcribed.

    Returns:
        dict: Completed transcript object.
    """
    # Set the API endpoint for creating a new transcript
    url = "https://api.assemblyai.com/v2/transcript"

    # Set the headers for the request, including the API token and content type
    headers = {
        "authorization": api_token,
        "content-type": "application/json"
    }

    # Set the data for the request, including the URL of the audio file to be transcribed
    data = {
        "audio_url": audio_url,
        "speaker_labels": True
    }

    # Send a POST request to the API to create a new transcript, passing in the headers and data
    response = requests.post(url, json=data, headers=headers)

    # Get the transcript ID from the response JSON data
    transcript_id = response.json()['id']

    # Set the polling endpoint URL by appending the transcript ID to the API endpoint
    polling_endpoint = f"https://api.assemblyai.com/v2/transcript/{transcript_id}"

    # Keep polling the API until the transcription is complete
    while True:
        # Send a GET request to the polling endpoint, passing in the headers
        transcription_result = requests.get(polling_endpoint, headers=headers).json()

        # If the status of the transcription is 'completed', exit the loop
        if transcription_result['status'] == 'completed':
            break

        # If the status of the transcription is 'error', raise a runtime error with the error message
        elif transcription_result['status'] == 'error':
            raise RuntimeError(f"Transcription failed: {transcription_result['error']}")

        # If the status of the transcription is not 'completed' or 'error', wait for 3 seconds and poll again
        else:
            time.sleep(3)

    return transcription_result

your_api_token = "bc7e213c3d14435d9f1e198ce37badb2"

# Upload a local file
filename = "/content/drive/MyDrive/AI_ML_INTERN /Task 3 Practice /vitals1.mp3"
upload_url = upload_file(your_api_token, filename)

# Transcribe it
transcrSpeaker = create_transcript(your_api_token, upload_url)

# Iterate through the utterances and print speaker information
print("\nSpeakers' Information:\n\n")

for utterance in transcrSpeaker['utterances']:
    speaker = utterance['speaker']
    text = utterance['text']
    confidence = utterance['confidence']

    print(f"Speaker {speaker}: {text}\nConfidence: {confidence}\n")


Speakers' Information:


Speaker A: Good morning, Mr. Anderson. How are you feeling today Morning? I'm feeling a bit tired, but overall not too bad. Alright, I'm just going to take a few of your vital signs. I'll start with your blood pressure. Could you roll up your sleeves please? Sure thing. Thank you. Okay, just stay relaxed. Here we go. Your blood pressure is 120 over 80, which is in a good range. Now let me check your pulse. I'll place my finger here on your wrist. No problem. That's a pulse of 72 beats per minute, which is within the normal range. Now I'll check your temperature. I'll place this thermometer under your tongue so that please keep your mouth closed around it. Great. Your temperature is 98.6 degrees Fahrenheit. Right on the target. Finally, I'll check your oxygen levels with this little device. I'll just clip it on your finger. Okay? Perfect. Your oxygen level is at 98%. Everything looks good. Med standards in. Let me know if you start feeling any discomfort or hav

In [ ]:
print(transcrSpeaker['utterances'])

[{'speaker': 'A', 'text': "Good morning, Mr. Anderson. How are you feeling today Morning? I'm feeling a bit tired, but overall not too bad. Alright, I'm just going to take a few of your vital signs. I'll start with your blood pressure. Could you roll up your sleeves please? Sure thing. Thank you. Okay, just stay relaxed. Here we go. Your blood pressure is 120 over 80, which is in a good range. Now let me check your pulse. I'll place my finger here on your wrist. No problem. That's a pulse of 72 beats per minute, which is within the normal range. Now I'll check your temperature. I'll place this thermometer under your tongue so that please keep your mouth closed around it. Great. Your temperature is 98.6 degrees Fahrenheit. Right on the target. Finally, I'll check your oxygen levels with this little device. I'll just clip it on your finger. Okay? Perfect. Your oxygen level is at 98%. Everything looks good. Med standards in. Let me know if you start feeling any discomfort or have any conc

***Summarization***

In [ ]:
import requests
import json
import time

def read_file(filename, chunk_size=5242880):
    with open(filename, 'rb') as _file:
        while True:
            data = _file.read(chunk_size)
            if not data:
                break
            yield data

def upload_file(api_token, path):
    """
    Upload a file to the AssemblyAI API.

    Args:
        api_token (str): Your API token for AssemblyAI.
        path (str): Path to the local file.

    Returns:
        str: The upload URL.
    """
    headers = {'authorization': api_token}
    response = requests.post('https://api.assemblyai.com/v2/upload',
                             headers=headers,
                             data=read_file(path))

    if response.status_code == 200:
        return response.json()["upload_url"]
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

def create_transcript(api_token, audio_url):
    """
    Create a transcript using AssemblyAI API.

    Args:
        api_token (str): Your API token for AssemblyAI.
        audio_url (str): URL of the audio file to be transcribed.

    Returns:
        dict: Completed transcript object.
    """
    # Set the API endpoint for creating a new transcript
    url = "https://api.assemblyai.com/v2/transcript"

    # Set the headers for the request, including the API token and content type
    headers = {
        "authorization": api_token,
        "content-type": "application/json"
    }

    # Set the data for the request, including the URL of the audio file to be transcribed
    data = {
        "audio_url": audio_url,
        "summarization": True,
        "summary_model": "informative",
        "summary_type": "bullets"
    }

    # Send a POST request to the API to create a new transcript, passing in the headers and data
    response = requests.post(url, json=data, headers=headers)

    # Get the transcript ID from the response JSON data
    transcript_id = response.json()['id']

    # Set the polling endpoint URL by appending the transcript ID to the API endpoint
    polling_endpoint = f"https://api.assemblyai.com/v2/transcript/{transcript_id}"

    # Keep polling the API until the transcription is complete
    while True:
        # Send a GET request to the polling endpoint, passing in the headers
        transcription_result = requests.get(polling_endpoint, headers=headers).json()

        # If the status of the transcription is 'completed', exit the loop
        if transcription_result['status'] == 'completed':
            break

        # If the status of the transcription is 'error', raise a runtime error with the error message
        elif transcription_result['status'] == 'error':
            raise RuntimeError(f"Transcription failed: {transcription_result['error']}")

        # If the status of the transcription is not 'completed' or 'error', wait for 3 seconds and poll again
        else:
            time.sleep(3)

    return transcription_result

your_api_token = "bc7e213c3d14435d9f1e198ce37badb2"

# Upload a local file
filename = "/content/drive/MyDrive/AI_ML_INTERN /Task 3 Practice /vitals1.mp3"
upload_url = upload_file(your_api_token, filename)

# Transcribe it
transcript = create_transcript(your_api_token, upload_url)

# Print the summary
print(transcript["summary"])

- Good morning, Mr. Anderson. How are you feeling today Morning? I'm feeling a bit tired, but overall not too bad. I'm just going to take a few of your vital signs. Let me know if you start feeling any discomfort or have any concerns.


***Translate the text from english to any language the user want***

In [ ]:
from deep_translator import GoogleTranslator

translator = GoogleTranslator(source='auto', target='fr')


# Split the text into smaller chunks
chunk_size = 500  # Adjust as per your needs
chunks = [result[i:i+chunk_size] for i in range(0, len(result), chunk_size)]

# Translate each chunk and store the results
translations = []
for chunk in chunks:
    translation = translator.translate(chunk)
    translations.append(translation)

# Concatenate the translations
translated_text = " ".join(translations)

print(translated_text)



Bonjour, M. Anderson. Comment vous sentez-vous aujourd'hui ? Je me sens un peu fatigué, mais dans l'ensemble, ça ne va pas trop mal. Bon, je vais juste prendre quelques-uns de vos signes vitaux. Je vais commencer par votre tension artérielle. Pourriez-vous retrousser vos manches, s'il vous plaît ? Bien sûr. Merci. Bon, restez détendu. C'est parti. Votre tension artérielle est de 120 sur 80, ce qui est dans une bonne fourchette. Maintenant, laissez-moi vérifier votre pouls. Je vais placer mon doigt ici sur votre poignet. Pas de problème. C'est un pouls de 72 battements par minute, ce qui est dans la plage normale. Maintenant, je vais vérifier votre température. Je vais placer ce thermomètre sous votre langue afin de garder votre bouche fermée autour. Super. Votre température est de 98,6 degrés Fahrenheit. Exactement la cible. Enfin, je vais vérifier votre taux d'oxygène avec ce petit appareil. Je vais juste le clipser sur votre doigt. D'accord ? Parfait. Votre taux d'oxygène est à 98 %.

In [ ]:
from deep_translator import GoogleTranslator

translator = GoogleTranslator(source='auto', target='de')


# Split the text into smaller chunks
chunk_size = 500  # Adjust as per your needs
chunks = [transcript["summary"][i:i+chunk_size] for i in range(0, len(transcript["summary"]), chunk_size)]

# Translate each chunk and store the results
translations = []
for chunk in chunks:
    translation = translator.translate(chunk)
    translations.append(translation)

# Concatenate the translations
translated_text = " ".join(translations)

print(translated_text)



- Guten Morgen, Mr. Anderson. Wie fühlen Sie sich heute Morgen? Ich fühle mich ein bisschen müde, aber insgesamt nicht so schlecht. Ich werde nur ein paar Ihrer Vitalfunktionen messen. Sagen Sie mir Bescheid, wenn Sie sich unwohl fühlen oder irgendwelche Bedenken haben.


In [ ]:
# Split the transcript into chunks of maximum length 80 characters
chunks = [translated_text[i:i+110] for i in range(0, len(translated_text), 110)]

# Join the chunks with line breaks
formatted_text_Trans= "\n".join(chunks)

print(formatted_text_Trans)

- Guten Morgen, Mr. Anderson. Wie fühlen Sie sich heute Morgen? Ich fühle mich ein bisschen müde, aber insgesa
mt nicht so schlecht. Ich werde nur ein paar Ihrer Vitalfunktionen messen. Sagen Sie mir Bescheid, wenn Sie si
ch unwohl fühlen oder irgendwelche Bedenken haben.


In [ ]:
from deep_translator import GoogleTranslator

translator = GoogleTranslator(source='auto', target='mr')

# Iterate through the utterances and print translated speaker information
print("\nInformation des intervenants:\n\n")

for utterance in transcrSpeaker['utterances']:
    speaker = utterance['speaker']
    text = utterance['text']
    confidence = utterance['confidence']

    # Translate the speaker and text
    translated_speaker = translator.translate(speaker)
    translated_text = translator.translate(text)

    print(f"Intervenant {translated_speaker}: {translated_text}\nConfiance: {confidence}\n")



Information des intervenants:


Intervenant ए: सुप्रभात, मिस्टर अँडरसन. आज सकाळ कशी वाटत आहे? मला थोडं थकल्यासारखं वाटतंय, पण एकंदरीत फार वाईट नाही. ठीक आहे, मी फक्त तुमची काही महत्वाची चिन्हे घेणार आहे. मी तुमच्या रक्तदाबाने सुरुवात करेन. कृपया तुम्ही तुमची बाही गुंडाळू शकाल का? नक्की गोष्ट. धन्यवाद. ठीक आहे, फक्त आरामशीर राहा. येथे आम्ही जातो. तुमचा रक्तदाब 80 च्या वर 120 आहे, जो चांगल्या श्रेणीत आहे. आता मला तुमची नाडी तपासू द्या. मी माझे बोट इथे तुझ्या मनगटावर ठेवतो. हरकत नाही. ती 72 बीट्स प्रति मिनिटाची नाडी आहे, जी सामान्य श्रेणीमध्ये आहे. आता मी तुमचे तापमान तपासेन. मी हे थर्मामीटर तुमच्या जिभेखाली ठेवेन जेणेकरून कृपया तुमचे तोंड त्याच्याभोवती बंद ठेवा. मस्त. तुमचे तापमान ९८.६ अंश फॅरेनहाइट आहे. अगदी लक्ष्यावर. शेवटी, मी या छोट्या उपकरणाने तुमची ऑक्सिजन पातळी तपासेन. मी फक्त ते तुमच्या बोटावर क्लिप करेन. ठीक आहे? परफेक्ट. तुमची ऑक्सिजन पातळी 98% आहे. सर्व काही चांगले दिसते. मेड स्टँडर्ड्स इन. तुम्हाला काही अस्वस्थता जाणवू लागल्यास किंवा काही चिंता असल्यास मला कळवा. धन्यवाद. मला